In [199]:
import csv
 
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [x for x in dataset[i]]
    return dataset

In [200]:
def convertCatToNum(dataset):
    for row in dataset:
        if (row[0] == 'vhigh'): row[0] = 1;
        elif (row[0] == 'high'): row[0] = 2;
        elif (row[0] == 'med'): row[0] = 3;
        elif (row[0] == 'low'): row[0] = 4;
        
        if (row[1] == 'vhigh'): row[1] = 1;
        elif (row[1] == 'high'): row[1] = 2;
        elif (row[1] == 'med'): row[1] = 3;
        elif (row[1] == 'low'): row[1] = 4;
        
        if (row[2] == '2'): row[2] = 2;
        elif (row[2] == '3'): row[2] = 3;
        elif (row[2] == '4'): row[2] = 4;
        elif (row[2] == '5'): row[2] = 5;
        elif (row[2] == 'more' or row[2] == '5more'): row[2] = 6;
        
        if (row[3] == '2'): row[3] = 2;
        elif (row[3] == '4'): row[3] = 4;
        elif (row[3] == 'more'): row[3] = 5;
        
        if (row[4] == 'small'): row[4] = 1;
        elif (row[4] == 'med'): row[4] = 2;
        elif (row[4] == 'big'): row[4] = 3;
        
        if (row[5] == 'low'): row[5] = 1;
        elif (row[5] == 'med'): row[5] = 2;
        elif (row[5] == 'high'): row[5] = 3;
        
        if (row[6] == 'unacc'): row[6] = 1;
        elif (row[6] == 'acc'): row[6] = 2;
        elif (row[6] == 'good'): row[6] = 3;
        elif (row[6] == 'vgood'): row[6] = 4;

In [201]:
import random

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [202]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [203]:
import math

def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [204]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [205]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [206]:
def calculateProbability(x, mean, stdev):
    
    if stdev == 0:
        return 1. if x == mean else .1
    
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * math.pow(stdev, 2))) * exponent

In [207]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [208]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [209]:
carDS = loadCsv("carData.csv")

In [210]:
convertCatToNum(carDS)

In [266]:
splittedCarDS = splitDataset(carDS, 0.70)

In [267]:
trainSet = splittedCarDS[0]
testSet = splittedCarDS[1]

In [268]:
summary = summarizeByClass(trainSet)

In [269]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [270]:
predictions = getPredictions(summary, testSet)

In [271]:
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [272]:
accuracy = getAccuracy(testSet, predictions)

In [273]:
accuracy

70.52023121387283